<a href="https://colab.research.google.com/github/Chris-Congleton/MSc-Thesis/blob/main/Volkskrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and loading data

In [53]:
import json, os
import pandas as pd
import nltk
import glob
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm
nltk.download('punkt')
import seaborn as sns
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [54]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [55]:
path_to_json = "gdrive/MyDrive/CS/Thesis/volkskrant-nl"
json_files = ["gdrive/MyDrive/CS/Thesis/volkskrant-nl/"+pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
print(json_files[:5])

['gdrive/MyDrive/CS/Thesis/volkskrant-nl/volkskrant-nl-2002-01.json', 'gdrive/MyDrive/CS/Thesis/volkskrant-nl/volkskrant-nl-2002-02.json', 'gdrive/MyDrive/CS/Thesis/volkskrant-nl/volkskrant-nl-2002-03.json', 'gdrive/MyDrive/CS/Thesis/volkskrant-nl/volkskrant-nl-2002-04.json', 'gdrive/MyDrive/CS/Thesis/volkskrant-nl/volkskrant-nl-2002-05.json']


In [56]:
def readFiles(file_names):
  dfs = [] # an empty list to store the data frames
  for f in tqdm(file_names):
    data = pd.read_json(f) # read data frame from json file
    dfs.append(data) # append the data frame to the list

  df = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list.
  return df

In [ ]:
df = readFiles(json_files)
df.shape

 55%|█████▍    | 117/214 [00:27<00:56,  1.73it/s]

In [ ]:
df.tail(2)

# Exploratory Data Analysis

In [ ]:
f, axs = plt.subplots(1,figsize=(10, 8))
sns.histplot(data=df, x="categories", hue="categories", ax=axs)
axs.tick_params(axis='x', rotation=45)
axs.set_title("Total number of articles")

In [ ]:
df_year = df.copy(deep=True)
df_year["year"] = df_year.timestamp.dt.year
time_df = pd.DataFrame({'count':df_year.groupby(["year","categories"]).size()}).reset_index()
f, axs = plt.subplots(1,figsize=(20, 16))
sns.lineplot(data=time_df, x='year',y='count',hue='categories',ci=None, palette=["C0", "C1", "C2", "C3", "C4",
                                                                                 "C5", "C6", "C7", "C8", "C9"])
axs.set_title("Number of articles")

The counts of articles collected per year per category vary quite a bit with for example ~45.000 "News & Background" articles in 2012 and ~10.000 in 2019. These should be normalised when counting party or politician mentions for example. 

In [ ]:
df_nwords = df.copy(deep=True)
df_nwords["year"] = df_nwords.timestamp.dt.year
df_nwords["nwords"] = df_nwords.cleantext.str.len()
df_nwords = pd.DataFrame(df_nwords.groupby(["year","categories"])["nwords"].agg("sum")).reset_index()

f, axs = plt.subplots(1,figsize=(20, 16))
sns.lineplot(data=df_nwords, x='year',y='nwords',hue='categories',ci=None, palette=["C0", "C1", "C2", "C3", "C4",
                                                                                 "C5", "C6", "C7", "C8", "C9"])
axs.set_title("Total length of articles")

The total amount of words collected is highest for news and background articles. This graph shows a different balance in data.

# Frequentie politieke partijen

In [ ]:
# Lijst partijen tweede kamer per jaar: 
# Bron: https://www.parlement.com/id/vk9scmucgjhj/aantal_deelnemende_partijen_tweede

tk_2002 = ["CDA", "LPF", "VVD", "PvdA", "GL", "SP", "D66", "CU", "SGP", "LN"]
tk_2003 = ["CDA", "LPF", "VVD", "PvdA", "GnL", "SP", "D66", "CU", "SGP"]
tk_2006 = ["CDA", "VVD", "PvdA", "GL", "SP", "D66", "CU", "SGP", "PVV", "PvdD"]
tk_2010 = ["CDA", "VVD", "PvdA", "GL", "SP", "D66", "CU", "SGP", "PVV", "PvdD"]
tk_2012 = ["CDA", "VVD", "PvdA", "GL", "SP", "D66", "CU", "SGP", "PVV", "PvdD", "50PLUS"]
tk_2017 = ["CDA", "VVD", "PvdA", "GL", "SP", "D66", "CU", "SGP", "PVV", "PvdD", "50PLUS", "DENK", "FVD"]
tk_2021 = ["CDA", "VVD", "PvdA", "GL", "SP", "D66", "CU", "SGP", "PVV", "PvdD", "50PLUS", "DENK",
           "FVD", "BIJ1", "BBB", "JA21", "VOLT"]

# Convert to one unique list
tk = [tk_2002, tk_2003,tk_2006,tk_2010,tk_2012,tk_2017,tk_2021]
tk = [item for sublist in tk for item in sublist]
partijen = list(set(tk))
print(partijen)

In [ ]:
df_partij = df.copy(deep=True)
df_partij["year"] = df_partij.timestamp.dt.year

for p in partijen:
  df_partij[p] = df_partij.cleantext.str.count(p)

df_party_counts = pd.DataFrame()

for p in partijen:
  df_party_counts[p] = df_partij.groupby(["year"])[p].agg("sum")

df_party_counts = df_party_counts.reset_index()
df_mentions = df_party_counts.melt('year')

In [ ]:
df_mentions = df_mentions.rename(columns={'year':'Year','variable':'Party','value':'Frequency'})

# Drop rows where frequency is <1000
df_mentions = df_mentions[df_mentions.Frequency > 1000]
print(df_mentions.head())

In [ ]:
f, axs = plt.subplots(1,figsize=(20, 16))
sns.lineplot(data=df_mentions, x='Year',y='Frequency', hue='Party')
axs.set_title("Party mentions in Volkskrant")

# Frequentie politici

In [ ]:
politici = ["Kees van der Staaij", "Sigrid Kaag", "Lilian Marijnissen", "Geert Wilders", "Wopke Hoekstra", "Lilianne Ploumen", "Mark Rutte"]

In [ ]:
# Computing number of articles per year to normalise counts
df_norm = time_df.groupby("year").sum("count").reset_index()
df_norm = df_norm.rename(columns={"year":"Year", "count":"Articles"})
print(df_norm.head())

In [ ]:
# Count occurence of politicians
df_politici = df.copy(deep=True)
df_politici["year"] = df_politici.timestamp.dt.year

for p in politici:
  df_politici[p] = df_politici.cleantext.str.count(p)

df_politici_counts = pd.DataFrame()

for p in politici:
  df_politici_counts[p] = df_politici.groupby(["year"])[p].agg("sum")

df_politici_counts = df_politici_counts.reset_index()
df_pol_mentions = df_politici_counts.melt('year')

df_pol_mentions = df_pol_mentions.rename(columns={'year':'Year','variable':'Politician','value':'Frequency'})

df_pol_mentions.tail()

In [ ]:
df_pol_mentions = df_pol_mentions.merge(df_norm, on="Year")
print(df_pol_mentions.head())

In [ ]:
# Drop rows where frequency is <1000
# df_pol_mentions = df_pol_mentions[df_pol_mentions.Frequency > 1000]
# print(df_pol_mentions.head())
df_pol_mentions["Normalised Freq"] = df_pol_mentions["Frequency"]/df_pol_mentions["Articles"]*10000
print(df_pol_mentions.head())

f, axs = plt.subplots(1,figsize=(20, 16))
sns.lineplot(data=df_pol_mentions, x='Year',y='Normalised Freq', hue='Politician')
axs.set_title("Politician mentions in Volkskrant (normalised)")

In [ ]:
from gensim.parsing.preprocessing import preprocess_string
from gensim import utils
import gensim.models

In [ ]:
volkskrant_corpus = list(df["cleantext"].values[1000])
print(len(volkskrant_corpus))

In [ ]:
class VolkskrantCorpus():
    """An iterator that yields sentences (lists of str)."""

    def __init__(self, corpus):
      self.corpus = corpus

    def __iter__(self):
        for line in self.corpus:
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [ ]:
sentences = VolkskrantCorpus(volkskrant_corpus)
model = gensim.models.Word2Vec(sentences=sentences)

In [ ]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling


def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    # extract the words & their vectors, as numpy arrays
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  # fixed-width numpy strings

    # reduce using t-SNE
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)

In [ ]:
import tempfile

with tempfile.NamedTemporaryFile(prefix='gensim-model-', delete=False) as tmp:
    temporary_filepath = tmp.name
    model.save(temporary_filepath)
    # The model is now safely stored in the filepath.
    # You can copy it to other machines, share it with others, etc.

In [ ]:
# To load a saved model:
new_model = gensim.models.Word2Vec.load(temporary_filepath)